In [1]:
import pandas as pd
import numpy as np
import keras

Using TensorFlow backend.


In [2]:
#use GPU instead of CPU
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14880231406197852337
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3059115622
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3000915817517306173
physical_device_desc: "device: 0, name: GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [3]:
import pickle
df= pd.read_pickle("./df_balanced_new.pkl")
df

,Class,description
0,EC002395,I4017(D933)8.8 6K-TAPBOUT M24X110
1,EC002395,I4017(D933)8.8 6K-TAPBOUT M24X320
2,EC002395,I4017(D933) 8.8 ELVZ 6K-TAPBOUT M6X30
3,EC002395,I4017(D933) 8.8 GLP 6K-TAPBOUT M3X12
4,EC002395,"I8676 8.8 ELVZ 6K-TAPBOUT MEF M12X1,25X40"
...,...,...
154585,EC001140,"Middenspanningskabel, XLPE geïsoleerd, alumini..."
154586,EC001140,"Middenspanningskabel, XLPE geïsoleerd, alumini..."
154587,EC001140,"Middenspanningskabel, XLPE geïsoleerd, alumini..."
154588,EC001140,"Middenspanningskabel, XLPE geïsoleerd, alumini..."


In [4]:
X = df["description"]
Y = df["Class"]

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
#vectorization of description
li = X.values.tolist()
vectorizer = TfidfVectorizer()
training = vectorizer.fit_transform(li)
print(vectorizer.get_feature_names())

['00', '000', '0000', '0000000', '00000006200', '00000006d00', '00000006f00', '00000006l00', '00000008007s', '000000080g2', '000000090em', '000000090ep', '000000090s1', '0000000c1fz', '0000000c3fz', '0000000dh00', '0000000g400', '0000000l700', '0000000q0du', '0000000q1ep', '0000000t0q1', '0000000wa00', '0000000yp00', '0000000z216', '0000001', '0000002', '0000003', '00001', '00002', '00003', '00004', '00005', '00006', '00007', '00008', '0001', '00010', '00011', '00012', '00013', '0002', '00020', '00021', '00022', '00023', '00024', '00025', '00026', '00027', '00028', '00029', '0003', '00030921', '00032', '00033', '00034', '00036', '00038', '0004', '00040101', '00040102', '00042', '00048', '00050', '00051', '0006', '00060', '00064', '00065', '00067', '00068', '00076', '00099', '000ae', '000ar', '000mbit', '000mm', '000x40', '001', '0010', '0010026813', '0010026814', '0010026815', '0010026816', '0010026817', '00101', '00102', '00104', '00106', '00107', '00111', '00112', '00114', '00115', '

In [7]:
print(training)

  (0, 65559)	0.5826106911767465
  (0, 82672)	0.4075833542327147
  (0, 29766)	0.3014533054508606
  (0, 46763)	0.4478628329894946
  (0, 58089)	0.4505389875896841
  (1, 65584)	0.5751099950063444
  (1, 82672)	0.410253629807203
  (1, 29766)	0.303428271773785
  (1, 46763)	0.4507969988999359
  (1, 58089)	0.45349068628253236
  (2, 66117)	0.5072764116213115
  (2, 51137)	0.25852484914431917
  (2, 82672)	0.41226763853328596
  (2, 29766)	0.30491785563774704
  (2, 46763)	0.45301004230409336
  (2, 58089)	0.45571695348167834
  (3, 65788)	0.5110241377188537
  (3, 55103)	0.38870418242279914
  (3, 82672)	0.3844678213164059
  (3, 29766)	0.2843567932098311
  (3, 46763)	0.42246290448297114
  (3, 58089)	0.4249872846323461
  (4, 13626)	0.4600953987676734
  (4, 65230)	0.40606986257538397
  (4, 66941)	0.4447800365395803
  :	:
  (154587, 63001)	0.21712560144432205
  (154587, 51264)	0.1327706116357527
  (154587, 58471)	0.10034059456010859
  (154588, 63164)	0.2927971610760937
  (154588, 18866)	0.278625697794167
 

In [8]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import precision_score, accuracy_score, recall_score
from sklearn.decomposition import TruncatedSVD


In [9]:
Y

0         EC002395
1         EC002395
2         EC002395
3         EC002395
4         EC002395
            ...   
154585    EC001140
154586    EC001140
154587    EC001140
154588    EC001140
154589    EC001140
Name: Class, Length: 154590, dtype: object

In [10]:
from sklearn.model_selection import train_test_split
SEED = 2000
x_train, x_test, y_train, y_test = train_test_split(training, Y, test_size = 0.1, random_state = SEED, stratify = Y)

In [11]:
from sklearn.metrics import precision_score, accuracy_score, recall_score


In [12]:
#Naive Bayes classifier
def test_NaiveBayes(x_train, x_test, y_train, y_test):
    MNB = MultinomialNB()
    NBClassifier = MNB.fit(x_train, y_train)
    predictions = NBClassifier.predict(x_test)
    a = accuracy_score(y_test, predictions)
    p = precision_score(y_test, predictions, average = 'weighted')
    r = recall_score(y_test, predictions, average = 'weighted')
    return a, p, r

In [13]:
#Multilayer Perceptron classfier
def test_NN(x_train, x_test, y_train, y_test):
    NN = MLPClassifier(solver = 'lbfgs', alpha = 0.00095, learning_rate = 'adaptive', learning_rate_init = 0.005, max_iter = 300, random_state = 0)
    Perceptron = NN.fit(x_train, y_train)
    predictions = Perceptron.predict(x_test)
    a = accuracy_score(y_test, predictions)
    p = precision_score(y_test, predictions, average = 'weighted')
    r = recall_score(y_test, predictions, average = 'weighted')
    return a, p, r

In [14]:
#SVM classifier
def test_SVM(x_train, x_test, y_train, y_test):
    SVM = SVC(kernel = 'linear')
    SVMClassifier = SVM.fit(x_train, y_train)
    predictions = SVMClassifier.predict(x_test)
    a = accuracy_score(y_test, predictions)
    p = precision_score(y_test, predictions, average = 'weighted')
    r = recall_score(y_test, predictions, average = 'weighted')
    return a, p, r

In [15]:
#SGD classifier
#classifier that minimizes the specified loss using stochastic gradient descent
def test_SGD(x_train, x_test, y_train, y_test):
    SGD = SGDClassifier(loss = 'modified_huber')
    SGDC = SGD.fit(x_train, y_train)
    predictions = SGDC.predict(x_test)
    a = accuracy_score(y_test, predictions)
    p = precision_score(y_test, predictions, average = 'weighted')
    r = recall_score(y_test, predictions, average = 'weighted')
    return a, p, r

In [16]:
#Voting Classifiers
def test_voting(x_train, x_test, y_train, y_test):
    SVM = SVC(kernel = 'linear', probability = True)
    SGD = SGDClassifier(loss = 'modified_huber')
    EnsembleClassifier = VotingClassifier(estimators = [('sgd', SGD), ('svc', SVM)], voting = 'soft', weights = [1,1])
    EnsembleClassifier = EnsembleClassifier.fit(x_train, y_train)
    predictions = EnsembleClassifier.predict(x_test)
    a = accuracy_score(y_test, predictions)
    p = precision_score(y_test, predictions, average = 'weighted')
    r = recall_score(y_test, predictions, average = 'weighted')
    return a, p, r

In [17]:
accuracy, precision, recall = test_SGD(x_train, x_test, y_train, y_test)
print(accuracy)

C:\Users\BI\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.8364059771007181


In [ ]:
accuracy, precision, recall = test_voting(x_train, x_test, y_train, y_test)
print(accuracy)

In [ ]:
accuracy, precision, recall = test_NaiveBayes(x_train, x_test, y_train, y_test)
print(accuracy)

In [ ]:
accuracy, precision, recall = test_NN(x_train, x_test, y_train, y_test)
print(accuracy)

In [ ]:
accuracy, precision, recall = test_SVM(x_train, x_test, y_train, y_test)
print(accuracy) 

In [ ]:
import pickle
pickle.dump(Perceptron, open('Perceptron_88', 'wb'))

In [ ]:
import pickle
pickle.dump(SGDC, open('SGDC_83', 'wb'))

In [ ]:
import pickle
pickle.dump(NBClassifier, open('NBClassifier_71', 'wb'))

In [ ]:
import pickle
pickle.dump(SVMClassifier, open('SVMClassifier_', 'wb'))

In [ ]:
import pickle
pickle.dump(EnsembleClassifier, open('EnsembleClassifier_', 'wb'))